In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

In [3]:
df = pd.read_fwf('Dataset/drf.txt')
data_csv_file = 'imd_data.csv'
df.to_csv(data_csv_file)

In [6]:
# remove the YEAR MN etc description in the line 12 from the csv file

In [7]:
whole_data = pd.read_csv(data_csv_file)
whole_data = whole_data.loc[:, ~whole_data.columns.str.match('Unnamed')]
whole_data.head()

,DAILY RAINFALL DATA :
0,-------------------
1,MN = MONTH
2,DRF01 = DAILY RAINFALL IN MM FOR DATE 01
3,DRF02 = DAILY RAINFALL IN MM FOR DATE 02
4,... ...


In [8]:
all_hyphen_indices = whole_data.index[whole_data['DAILY RAINFALL DATA :'] == '-'*224].tolist()

In [10]:
hyphen_indices_list = all_hyphen_indices[:-1]
station_bounds_list = all_hyphen_indices[1:]
print(len(hyphen_indices_list))
print(len(station_bounds_list))
# hyphen_indices

136
136


In [12]:
from iteration_utilities import grouper
header_indices = list(grouper(hyphen_indices_list, 2))
station_indices = list(grouper(station_bounds_list, 2))

In [13]:
all_stations = []
waste_chars = ['',',',' ']
stations_dic = {}
for index_tuple in header_indices:
    station_details = whole_data.iloc[index_tuple[0]+1]['DAILY RAINFALL DATA :']
    station_details_list = station_details.split(' ')
    pure_station_details = [x for x in station_details_list if x not in waste_chars]
    all_stations.append(pure_station_details)
    
    dist_ind = pure_station_details.index('Distt.')
    station_name = ''
    for i in range(0, dist_ind):
        station_name = station_name +" "+ pure_station_details[i]
    stations_dic[station_name] = {}
    catchment_ind = pure_station_details.index('CATCHMENT')
    dist_name = ''
    for i in range(dist_ind+2,catchment_ind):
        dist_name = dist_name +" "+pure_station_details[i]
    stations_dic[station_name]['Distt.'] = dist_name
    stations_dic[station_name]['Catchment_No.'] = pure_station_details[catchment_ind+3]
    lat_ind = pure_station_details.index('LATITUDE')
    lon_ind = pure_station_details.index('LONGITUDE')
    lat = ''
    lon = ''
    for i in range(lat_ind+2,lon_ind):
        lat = lat +" "+pure_station_details[i]
    for i in range(lon_ind+2,len(pure_station_details)):
        lon = lon +" "+pure_station_details[i]
    stations_dic[station_name]['lat'] = lat
    stations_dic[station_name]['lon'] = lon
    bound_indices = station_indices[header_indices.index(index_tuple)]
    stations_dic[station_name]['bounds'] = bound_indices
# print stations_dic
# print len(stations_dic)

In [14]:
columns_rename_dic = {0:'Year', 1:'Month', 2:'d1', 3:'d2',4:'d3', 5:'d4',6:'d5',7:'d6',8:'d7',9:'d8',10:'d9',11:'d10',
                     12:'d11',13:'d12',14:'d13',15:'d14',16:'d15',17:'d16',18:'d17',19:'d18',20:'d19',21:'d20',22:'d21',
                     23:'d22',24:'d23',25:'d24',26:'d25',27:'d26',28:'d27',29:'d28',30:'d29',31:'d30',32:'d31'}

In [20]:
bounds = stations_dic[' JALKOT']['bounds']
df = whole_data.iloc[bounds[0]+1:bounds[1]]
df = df.join(df['DAILY RAINFALL DATA :'].str.split(r' +', 32,expand=True).rename(columns=columns_rename_dic))


In [26]:
station_dfs = []
for station in stations_dic:
    bounds = stations_dic[station]['bounds']
    df = whole_data.iloc[bounds[0]+1:bounds[1]]
    df['District'] = stations_dic[station]['Distt.']
    df['Station'] = station.strip()
    df = df.join(df['DAILY RAINFALL DATA :'].str.split(r' +', 32,expand=True).rename(columns=columns_rename_dic))
    df.drop(columns=['DAILY RAINFALL DATA :'],inplace=True)
    station_dfs.append(df)

processed_df = pd.concat(station_dfs)

In [27]:
processed_df.index = range(0,len(processed_df))
processed_df.to_csv('Maharashtra_Daily_Rainfall.csv')
processed_df.head()

,District,Station,Year,Month,d1,d2,d3,d4,d5,d6,...,d22,d23,d24,d25,d26,d27,d28,d29,d30,d31
0,KOLHAPUR,KAPASHI,1934,01,.0,.0,.0,.0,.0,.0,...,.0,.0,.0,.0,.0,.0,.0,.0,.0,None
1,KOLHAPUR,KAPASHI,1934,02,.0,.0,.0,.0,.0,.0,...,.0,.0,.0,.0,.0,.0,None,None,None,None
2,KOLHAPUR,KAPASHI,1934,03,.0,.0,.0,.0,.0,.0,...,.0,.0,.0,.0,.0,.0,.0,.0,.0,None
3,KOLHAPUR,KAPASHI,1934,04,.0,.0,.0,.0,.0,.0,...,.0,.0,.0,.0,.0,.0,.0,.0,None,None
4,KOLHAPUR,KAPASHI,1934,05,.0,.0,.0,.0,.0,.0,...,.0,.0,.0,.0,.0,.0,.0,.0,.0,None


In [29]:
processed_df[processed_df.Station == "BHUDARGAD"]

,District,Station,Year,Month,d1,d2,d3,d4,d5,d6,...,d22,d23,d24,d25,d26,d27,d28,d29,d30,d31
41193,KOLHAPUR,BHUDARGAD,1963,01,.0,.0,.0,.0,.0,.0,...,.0,.0,.0,.0,.0,.0,.0,.0,.0,.0
41194,KOLHAPUR,BHUDARGAD,1963,02,.0,.0,.0,.0,.0,.0,...,.0,.0,.0,.0,.0,.0,.0,None,None,None
41195,KOLHAPUR,BHUDARGAD,1963,03,.0,.0,.0,.0,.0,.0,...,.0,.0,.0,.0,.0,5.1,.0,.0,14.8,.0
41196,KOLHAPUR,BHUDARGAD,1963,04,.0,.0,.0,.0,.0,.0,...,.0,.0,24.9,.0,.0,.0,.0,.0,.0,None
41197,KOLHAPUR,BHUDARGAD,1963,05,41.1,24.9,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
41198,KOLHAPUR,BHUDARGAD,1963,06,3.1,4.1,.0,.0,3.2,46.2,...,None,None,None,None,None,None,None,None,None,None
41199,KOLHAPUR,BHUDARGAD,1963,07,3.3,56.2,22.0,25.1,54.1,62.0,...,.4,6.0,.0,.0,6.3,3.3,4.0,14.1,4.1,23.3
41200,KOLHAPUR,BHUDARGAD,1963,08,13.1,.0,2.2,.0,37.1,13.3,...,19.0,7.0,18.6,3.8,24.2,8.2,8.0,3.0,1.2,.4
41201,KOLHAPUR,BHUDARGAD,1963,09,.0,7.2,.0,2.0,.0,3.0,...,None,None,None,None,None,None,None,None,None,None
41202,KOLHAPUR,BHUDARGAD,1963,10,.0,14.3,15.3,21.2,2.0,.0,...,None,None,None,None,None,None,None,None,None,None
